In [1]:
import pickle
import numpy as np
import pandas as pd
import os, glob
import seaborn as sns
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold, cross_validate
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

## The following are the ML models which can be used for trasinning
from sklearn.model_selection import RandomizedSearchCV
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from sklearn.linear_model import LinearRegression

from sklearn.preprocessing import MinMaxScaler,StandardScaler

import timeit
import warnings
warnings.filterwarnings("ignore")


ModuleNotFoundError: No module named 'lightgbm'

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl

import numpy as np
%matplotlib inline
import pandas as pd

In [ ]:
import seaborn as sns
sns.set(style="darkgrid")

In [ ]:
sns.set_context('talk')

In [ ]:
from IPython.display import display, HTML, Image

In [ ]:
import matplotlib.colors as mcolors

In [ ]:
dataFolder = os.getcwd()

In [ ]:
group1_low =1.634
group2_low =0.673
group2_upper =1.634
group3_upper =0.673

In [ ]:
## read the test file
data =pd.read_csv(os.path.join(dataFolder,'dataset5H_test.csv'))
data.columns =[col.strip() for col in data.columns]
data['ratio'] = data['b(CaO)']/data['b(SiO2)']
group1 = data[data['ratio']>=group1_low]  # with Portlandite, no Amor-S1
group2 = data[(data['ratio']<=group2_upper) & (data['ratio']>=group2_low)]  # no Portlandite, no Amor-S1
group3 = data[data['ratio']<=group3_upper]     # no Portlandite, with Amor-S1
groups = [group1,group2,group3]


In [ ]:
## read trained Model info:
modelInfo =pd.read_csv(os.path.join(dataFolder,'ModelTrainSummary.csv'))
modelInfo = modelInfo.iloc[:,1:4]
modelInfo

In [ ]:
modelInfo['modelType'].unique()

In [ ]:
modelInfo=modelInfo[modelInfo['modelType'].isin(['CONST','linear','GPyModel'])]

In [ ]:
modelTypeLst = modelInfo['modelType'].unique()
MLModels =[ml for ml in modelTypeLst if 'Model' in ml]
nonMLModels = [ml for ml in modelTypeLst if 'Model' not in ml]

In [ ]:
## load the mdoels
dataCols = data.columns.to_list()
modelsLst = []
for irow,row in modelInfo.iterrows():
    group = row['group']
    var =row['var']
    modelType = row['modelType']
    icol =dataCols.index(var)
    if '/' in var:
        fileCol = var.replace("/",'')
    else:
        fileCol = var            

    modelFolder = os.path.join(dataFolder,'SavedModel',group)
    if modelType=='CONST':
        ext = '.csv'
        fileName = os.path.join(modelFolder,modelType + '_'+str(icol)+'_'+fileCol+ext)
        tempModel =pd.read_csv(fileName)
    else:
        ext ='.sav'
        fileName = os.path.join(modelFolder,modelType + '_'+str(icol)+'_'+fileCol+ext)
        tempModel = pickle.load(open(fileName, 'rb'))
    
    modelsLst.append(tempModel)

modelInfo['modelObj'] = modelsLst   


In [ ]:
finalDF = pd.DataFrame()
for ML in MLModels:
    modelProcessing =[]
    modelProcessing = nonMLModels.copy()
    modelProcessing.append(ML)
    tempModelInfo = modelInfo[modelInfo['modelType'].isin(modelProcessing)]
    for col in group1.columns[4:-1]:  # exclude 'ratio' 
        colModelInfo = tempModelInfo[tempModelInfo['var'] == col]  
        
        for i, group in enumerate(groups):
        
            dataX = group.iloc[:,1:4]    
            dataY = group[col].values
            groupModelInfo = colModelInfo[colModelInfo['group'] =='group'+str(i+1)]
            modelType = groupModelInfo['modelType'].values[0]
            model = groupModelInfo['modelObj'].values[0]
            if modelType=='CONST':
                y_pred = list(model['const'].values)*len(dataY)
                
            elif modelType=='linear':
                y_pred = model.predict(dataX.values)
            
            else:
                scaler = StandardScaler().fit(dataX.values)
                X_scaled = scaler.transform(dataX.values)  # This will be used for input of trainning dataset
                y_pred  = model.predict(X_scaled)  
    
            tempDF = pd.DataFrame({'testDataY':dataY, 'predDataY':y_pred})
            tempDF['modelType'] = [modelType]*len(tempDF)
            tempDF['var'] = [col]*len(tempDF)
            tempDF['group'] = ['group'+str(i+1)]*len(tempDF)
            tempDF['modelSimulation'] = [ML+'_simulation']*len(tempDF)
                
            if len(finalDF)==0:   
                finalDF = tempDF
            else:
                finalDF = pd.concat([finalDF,tempDF],axis=0)
    

In [ ]:
finalDF.to_csv('Comparison_predict_testDataset.csv',index=False)

# Visualization of the prediction of the test dataset

In [ ]:
def drawPlots(trgVar,testDataY,predDataY,simulation):
    
    fig, ax =plt.subplots(figsize=(8,8))
    
    RMSE = mean_squared_error(testDataY,predDataY)
    R2   = r2_score(testDataY,predDataY)
    axMax = max(max(testDataY),max(predDataY))
    axMin = min(min(testDataY),min(predDataY))
    
    ## Fitting the line           
    x2= np.linspace(axMin,axMax,30);
    y2 = x2
    ax.plot(testDataY,predDataY,'bo',markerfacecolor = 'blue',markeredgecolor ='darkblue',markeredgewidth=0.5,label=trgVar)
    
    #ax.plot(df3['b(SiO2)'].values,df3['b(CaO)'].values,'bo',markerfacecolor = 'c',markeredgecolor ='b',markeredgewidth=0.5,label='no Amor-S1')
    #ax.plot(group3['b(SiO2)'].values,group3['b(CaO)'].values,'g^',markerfacecolor = 'm',markeredgecolor ='m',markeredgewidth=1,label='group3')
    ax.plot(x2,y2,'r-',lw=3,label ='1:1 ratio')
    ax.legend (loc='best',ncol=5)           
    ax.set_xlim(axMin,axMax)
    ax.set_ylim(axMin,axMax)
    #ax.text(0.05, 0.8, 'Above the line with Portlandite',fontsize = 20,color = 'k')
    #ax.text(0.4, 0.50, 'Below the line no Portlandite',fontsize = 20,color = 'k')
    ax.set_title(simulation+ ' ====> '+trgVar)
    ax.set_xlabel('testDataY')
    ax.set_ylabel('PredDataY')    
        
    return R2, RMSE    

In [ ]:
modelSimulations = finalDF['modelSimulation'].unique()
trgVars = finalDF['var'].unique()
statMeasure = []
for simulation in modelSimulations:
    if 'MLP' in simulation or 'xgb' in simulation: # skip the xgb and MLP models
        continue
    tempDF = finalDF[finalDF['modelSimulation']==simulation]
    for trgVar in trgVars:
        df = tempDF[tempDF['var']==trgVar]  # no Portlandite
        testDataY = df['testDataY'].values
        predDataY = df['predDataY'].values
        R2,RMSE = drawPlots(trgVar,testDataY,predDataY,simulation)
        statMeasure.append([trgVar,R2,RMSE,simulation])
    
statMeasDF = pd.DataFrame(statMeasure,columns=['Var','R2','RMSE','modelSimulation'])    


In [ ]:
statMeasDF

In [ ]:
statMeasDF_pivot=statMeasDF.pivot(index='Var',values=['R2','RMSE'],columns='modelSimulation')
statMeasDF.to_csv('statMeasure_trainDataset_noPivot.csv',index = False)

statMeasDF_pivot.to_csv('statMeasure_trainDataset_Pivot.csv')

In [ ]:
## alternative way to draw figures

In [ ]:
def drawCombiningPlots(plotDataDF,trgVar):
    
    fig, ax =plt.subplots(figsize=(10,10))
    
    marker = ['o', 'v', '^', '<', '>', '8', 's', 'p', '*', 'h', 'H', 'D', 'd', 'P', 'X']
    colors = ['tab:blue','tab:orange','tab:gray','tab:olive','tab:cyan','tab:green']
    ## Fitting the line           
    axisMin=1000
    axisMax =-1000
    for i, col in enumerate(plotDataDF.columns[1:]):
        testDataY = plotDataDF.iloc[:,0].values
        predDataY = plotDataDF[col].values
        axMax = max(max(testDataY),max(predDataY))
        axMin = min(min(testDataY),min(predDataY))
        axisMin = min(axisMin,axMin)
        axisMax = max(axisMax,axMax)
        ax.scatter(testDataY,predDataY,marker =marker[i],edgecolor =colors[i],label=col.replace('_simulation',''))

    x2= np.linspace(axisMin,axisMax,30);
    y2 = x2
    ax.plot(x2,y2,'r-',lw=3,label ='1:1 ratio')
    ax.legend (loc='best',ncol=3)           
    ax.set_xlim(axisMin,axisMax)
    ax.set_ylim(axisMin,axisMax)
    ax.set_title(trgVar)
    ax.set_xlabel('testDataY')
    ax.set_ylabel('PredDataY')    
        


In [ ]:
modelSimulations = finalDF['modelSimulation'].unique()
trgVars = finalDF['var'].unique()
for trgVar in trgVars:
    tempdf = finalDF[finalDF['var']==trgVar]  # no Portlandite
    dataplot = {}
    plotDataDF = pd.DataFrame()
    for simulation in modelSimulations:
        # skip the xgb and MLP models
        # the two models are very bad, need to be fine tune
        if 'MLP' in simulation or 'xgb' in simulation: 
            continue

        df = tempdf[tempdf['modelSimulation']==simulation]
        if len(plotDataDF)==0:
            plotDataDF=df[['testDataY','predDataY']]
        else:    
            plotDataDF = pd.concat([plotDataDF,df[['predDataY']]],axis=1)

        plotDataDF.rename({'predDataY': simulation}, axis=1,inplace=True)
                                    
    drawCombiningPlots(plotDataDF,trgVar)    
 
